In [14]:
import pandas as pd
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import json  # Se agrega la importación del módulo json

# Cargar los DataFrames desde los archivos CSV
movies = pd.read_csv(r'C:\Users\Jaime\Desktop\4GEEKS\PROYECTOS\021-PROYECTO1-k-nearest-neighbors-project-tutorial-main\tmdb_5000_movies.csv')
credits = pd.read_csv(r'C:\Users\Jaime\Desktop\4GEEKS\PROYECTOS\021-PROYECTO1-k-nearest-neighbors-project-tutorial-main\tmdb_5000_credits.csv')

# Combinar los DataFrames movies y credits en un nuevo DataFrame result_dataframe
result_dataframe = pd.merge(movies, credits, on='title')

# Función para cargar el contenido JSON de forma segura
def load_json_safe(json_str):
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return []

# Transformar los datos
result_dataframe['genres'] = result_dataframe['genres'].apply(lambda x: ' '.join([genre['name'] for genre in load_json_safe(x)]))
result_dataframe['keywords'] = result_dataframe['keywords'].apply(lambda x: ' '.join([keyword['name'] for keyword in load_json_safe(x)]))
result_dataframe['cast'] = result_dataframe['cast'].apply(lambda x: ' '.join([actor['name'] for actor in load_json_safe(x)[:3]]))
result_dataframe['crew'] = result_dataframe['crew'].apply(lambda x: ' '.join([crew_member['name'] for crew_member in load_json_safe(x) if crew_member['job'] == 'Director']))
result_dataframe['overview'] = result_dataframe['overview'].fillna('')  # Rellenar los valores NaN con cadena vacía
result_dataframe['overview'] = result_dataframe['overview'].apply(lambda x: x[0] if x else '')  # Obtener el resumen o cadena vacía

# Crear la columna 'tags' combinando las columnas relevantes
result_dataframe['tags'] = result_dataframe['genres'] + ' ' + result_dataframe['keywords'] + ' ' + result_dataframe['cast'] + ' ' + result_dataframe['crew'] + ' ' + result_dataframe['overview']

# Eliminar las columnas individuales que ya hemos combinado
result_dataframe.drop(columns=['genres', 'keywords', 'cast', 'crew', 'overview'], inplace=True)

# Crear una matriz TF-IDF para las etiquetas
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(result_dataframe['tags'])

# Construir el modelo KNN con k=10 (10 vecinos más cercanos)
knn_model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')
knn_model.fit(tfidf_matrix)

# Función para obtener las recomendaciones de películas basadas en una película de entrada
def get_movie_recommendations(movie_title):
    movie_index = result_dataframe[result_dataframe['title'] == movie_title].index[0]
    distances, indices = knn_model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(result_dataframe['title'][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]  # Excluimos la película de entrada (la más cercana)

# Prueba del sistema de recomendación ingresando una película
input_movie = "The Hunger Games"
recommendations = get_movie_recommendations(input_movie)
print("Recomendaciones para la película '{}'".format(input_movie))
for movie, distance in recommendations:
    print("Película: {}".format(movie))


Recomendaciones para la película 'The Hunger Games'
Película: The Hunger Games: Mockingjay - Part 1
Película: The Hunger Games: Mockingjay - Part 2
Película: The Hunger Games: Catching Fire
Película: Allegiant
Película: Cirque du Freak: The Vampire's Assistant
